In [30]:
#THIS IS JUST FOR TEST VISUALIZING. CHANGE CODE IN THE JSON PARSER.PY FILE

json = "/Users/skoebric/Dropbox/GitHub/gtgfoliummap/geometry/wherewework.geojson"
all_json = gpd.read_file(json)

m = folium.Map(width = '100%', height = 800, location = (20,5),zoom_start = 3,
               no_wrap=True,max_bounds=True, min_zoom=3, tiles="MapBox Bright")

style_function = lambda x: {'fillColor': '#73ad02',
                            'color': '#73ad02',
                            'weight':1,
                            }

def countryjsoner(row):
    country = row['Name']
    tk = row['Toolkit']
    link = row['Link']
    row_ = all_json.loc[all_json['Name'] == country]
    geojson = folium.GeoJson(row_, highlight_function = style_function, color = '#73ad02')
    popup = folium.Popup(f"<b>Country:</b> {country}<br>"
                         f"<b>Toolkit:</b> {tk}<br>"
                         f"<b><a href={link}> Link to Country Page</a></b>")
    popup.add_to(geojson)
    geojson.add_to(m)

all_json.apply(countryjsoner, axis = 1)
m


TypeError: ("__init__() got an unexpected keyword argument 'color'", 'occurred at index 0')

In [7]:
all_json[0:1]


,Toolkit,Name,geometry
0,Grid Integration,Vietnam,"(POLYGON ((104.06396484375 10.3908203125, 104...."


In [23]:
all_json.columns

Index(['Toolkit', 'Link', 'Name', 'geometry'], dtype='object')

In [ ]:
all_json.links
